In [1]:
import os
import subprocess

def create_tex_file(tex_content, filename="document.tex"):
    """Create a .tex file with the given content."""
    with open(filename, "w", encoding="utf-8") as tex_file:
        tex_file.write(tex_content)

def run_command(command):
    """Run a shell command with error handling."""
    try:
        subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {command}")
        print(e.stderr)
        exit(1)

def convert_tex_to_png(tex_content, output_png="document.png"):
    """Convert TeX content to a PNG file."""
    tex_filename = "document.tex"
    pdf_filename = "document.pdf"
    cropped_pdf = "document-cropped.pdf"

    print("Creating TeX file...")
    create_tex_file(tex_content, tex_filename)

    print("Converting TeX to PDF...")
    run_command(f"xelatex -interaction=nonstopmode -no-pdf {tex_filename}")
    run_command(f"xelatex -interaction=nonstopmode {tex_filename}")

    if not os.path.exists(pdf_filename):
        print("PDF was not created successfully.")
        return

    # print("Cropping the PDF...")
    # run_command(f"pdfcrop {pdf_filename} {cropped_pdf}")

    print("Converting cropped PDF to PNG...")
    run_command(f"pdftoppm {pdf_filename} -png -singlefile document")
    os.rename("document.png", output_png)

    print(f"PNG created successfully: {output_png}")

tex_content = r"""
\documentclass{article}
\usepackage{fontspec}
\usepackage{amsmath}
\usepackage{mathspec}
\usepackage[paperheight=5in,paperwidth=6in,margin=0.1in]{geometry}

\setmainfont{JaneAusten}
\setmathsfont(Digits,Latin){JaneAusten}

\begin{document}
\large Testing the Jane Austen font with mathematics

Here is some text in Jane Austen font, followed by a mathematical equation:

Let's try a simpler equation first:

\[ a + b = c \]

And a slightly more complex one:

\[ y = 2x + 5 \]

Some text math: $E = mc^2$

And here is more text with numbers and letters:

ABCDEFGHIJKLMNOPQRSTUVWXYZ\\
abcdefghijklmnopqrstuvwxyz\\
\[
y = \frac{\sqrt{x^3 + 4x^2}}{2x + 5} + \sqrt{\frac{3x^2 + 7}{x + 1}}
\]
1234567890 
\end{document}
"""

convert_tex_to_png(tex_content)


Creating TeX file...
Converting TeX to PDF...
Converting cropped PDF to PNG...
PNG created successfully: document.png


In [ ]:
import openai

font = "JaneAusten"
iterations = 3
client = openai.Client(api_key="sk-rc-gNqk64G9aEO7nnxOEnpvmw", base_url="https://fmapi.swissai.cscs.ch")


header = r"""
You have to start your answer with the following structure for the latex font :
\documentclass{article}
\usepackage{fontspec}
\usepackage{amsmath}
\usepackage{mathspec}
\usepackage[paperwidth=6in,margin=0.1in]{geometry}

\setmainfont{%s}
\setmathsfont(Digits,Latin){%s}

\begin{document}

""" % (font, font)

for i in range(1, iterations + 1):

    question = "Answer only in latex format : give an example of a student solution to a math exercise number %s with hard equations involving sqrt and power and a text explanation. the answer should be %s"% (i, i)

    request = question + header
    res = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct",
        messages=[
            {
                "content": request, 
                "role": "user",
            }
        ],
        stream=True,
    )

    answer = r""""""
    for chunk in res:
        if len(chunk.choices) > 0 and chunk.choices[0].delta.content:
            answer += chunk.choices[0].delta.content

    answer = answer[answer.find('\\'):]
    with open(f'generated_data/txt/answer_{i}.txt', 'w') as f:
        f.write(answer)
    
    print("Generated ", i)

Generated  0


In [14]:
with open('generated_data/txt/answer_0.txt', 'r') as file:
    answer_1 = file.read()


convert_tex_to_png(answer_1)


Creating TeX file...
Converting TeX to PDF...
Converting cropped PDF to PNG...
PNG created successfully: document.png
